In [36]:
from include import *

from calendar import timegm
from jwkest import JWKESTException, BadSignature
from jwkest.jwk import KEYS
from jwkest.jws import JWS
import base64
import json
import jwt
import datetime
import logging
import requests

In the cell below, please assign the access token to be inspected to the variable `access_token`.

In [37]:
access_token="\
eyJhbGciOiJFUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI3V1V0N09BYzg2T2otRldXVjVWQlFJLVdsQURxQWdET19la05QaFNJUXRrIn0.eyJleHAiOjE2Mzc4NDQ0MDUsImlhdCI6MTYzNzg0NDEwNSwiYXV0aF90aW1lIjoxNjM3ODQ0MTA1LCJqdGkiOiI0ODhmNTYyMC1kZjAzLTQwYmItYmNhOS0xOGNmMTg1OGRlMzQiLCJpc3MiOiJodHRwOi8va2V5Y2xvYWs6ODA4MC9hdXRoL3JlYWxtcy9kZW1vIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImFhZmQxZmJiLWQ0MjktNDI2Zi04YTIxLWM1MWFmMzkwY2FmNyIsInR5cCI6IkJlYXJlciIsImF6cCI6ImF1dGhvcml6YXRpb24tY29kZS1ncmFudC1jbGllbnQiLCJzZXNzaW9uX3N0YXRlIjoiYmQ3YmUwY2MtOTM3YS00MThiLTg4Y2EtNWM3ZWIwMjBkZjM1IiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwOi8vMTI3LjAuMC4xIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtZGVtbyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCIsInNpZCI6ImJkN2JlMGNjLTkzN2EtNDE4Yi04OGNhLTVjN2ViMDIwZGYzNSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiRmFsayBIYXJ0bWFubiIsInByZWZlcnJlZF91c2VybmFtZSI6ImZhbGsuaGFydG1hbm5Ac2Npb3RhLmlvIiwiZ2l2ZW5fbmFtZSI6IkZhbGsiLCJmYW1pbHlfbmFtZSI6IkhhcnRtYW5uIiwiZW1haWwiOiJmYWxrLmhhcnRtYW5uQHNjaW90YS5pbyJ9.BhZngsEL7sXg-T__e2j4bVZhsgo5GbH5Fd2NTulyuHBmoxe2JjKOf9W1oZvMmgvQcfOkVlU4PPwsBY4-bMjfHA"

In [38]:
[header, payload, sig] = access_token.split(".")

header_decoded = base64.urlsafe_b64decode(header + "="*divmod(len(header),4)[1]).decode("ascii")
payload_decoded = base64.urlsafe_b64decode(payload + "="*divmod(len(payload),4)[1]).decode("ascii")
print("header:")
json_pretty_print(header_decoded)
print("payload:")
json_pretty_print(payload_decoded)
print("signature:\n{}".format(sig))

header:
{
    "alg": "ES256",
    "kid": "7WUt7OAc86Oj-FWWV5VBQI-WlADqAgDO_ekNPhSIQtk",
    "typ": "JWT"
}
payload:
{
    "acr": "1",
    "allowed-origins": [
        "http://127.0.0.1"
    ],
    "aud": "account",
    "auth_time": 1637844105,
    "azp": "authorization-code-grant-client",
    "email": "falk.hartmann@sciota.io",
    "email_verified": true,
    "exp": 1637844405,
    "family_name": "Hartmann",
    "given_name": "Falk",
    "iat": 1637844105,
    "iss": "http://keycloak:8080/auth/realms/demo",
    "jti": "488f5620-df03-40bb-bca9-18cf1858de34",
    "name": "Falk Hartmann",
    "preferred_username": "falk.hartmann@sciota.io",
    "realm_access": {
        "roles": [
            "offline_access",
            "default-roles-demo",
            "uma_authorization"
        ]
    },
    "resource_access": {
        "account": {
            "roles": [
                "manage-account",
                "manage-account-links",
                "view-profile"
            ]
        }
  

In [39]:
config_url = 'http://keycloak:8080/auth/realms/demo/.well-known/openid-configuration/'
oidc_config = requests.get(config_url, verify=True).json()
issuer = oidc_config['issuer']
print("Expected issuer {}".format(issuer))

jwks_keys: KEYS = KEYS()
jwks_keys.load_from_url(oidc_config['jwks_uri'])

try:
    content = JWS().verify_compact(access_token, keys=jwks_keys)
    print("Actual issuer {}".format(content["iss"]))
except BadSignature as e:
    print("Bad signature encountered.")
    raise e
    
    
utc_timestamp = timegm(datetime.datetime.utcnow().utctimetuple())
invalid = False
if utc_timestamp > content.get('exp', 0):
    print("Invalid Authorization header. JWT has expired.")
    invalid = True
if 'nbf' in content and utc_timestamp < content['nbf']:
    print("Invalid Authorization header. JWT not yet valid.")
    invalid = True
if not invalid:
    print("JWT is currently valid.")

print("\nToken content")
json_pretty_print(json.dumps(content))


    

    
    
    

Expected issuer http://keycloak:8080/auth/realms/demo
Actual issuer http://keycloak:8080/auth/realms/demo
JWT is currently valid.

Token content
{
    "acr": "1",
    "allowed-origins": [
        "http://127.0.0.1"
    ],
    "aud": "account",
    "auth_time": 1637844105,
    "azp": "authorization-code-grant-client",
    "email": "falk.hartmann@sciota.io",
    "email_verified": true,
    "exp": 1637844405,
    "family_name": "Hartmann",
    "given_name": "Falk",
    "iat": 1637844105,
    "iss": "http://keycloak:8080/auth/realms/demo",
    "jti": "488f5620-df03-40bb-bca9-18cf1858de34",
    "name": "Falk Hartmann",
    "preferred_username": "falk.hartmann@sciota.io",
    "realm_access": {
        "roles": [
            "offline_access",
            "default-roles-demo",
            "uma_authorization"
        ]
    },
    "resource_access": {
        "account": {
            "roles": [
                "manage-account",
                "manage-account-links",
                "view-profil